In [2568]:
# Importing necessary libraries and functions
import pandas
import numpy
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.stats import norm
from scipy.interpolate import PchipInterpolator, UnivariateSpline

In [2584]:
df = pandas.read_excel("raw_out_of_sample_data.xlsx")

In [2585]:
# Loading data required
call_df = df[['[QUOTE_UNIXTIME]', ' [QUOTE_READTIME]', ' [QUOTE_DATE]',
       ' [QUOTE_TIME_HOURS]', ' [UNDERLYING_LAST]', ' [EXPIRE_DATE]',
       ' [EXPIRE_UNIX]', ' [DTE]', ' [C_DELTA]', ' [C_GAMMA]', ' [C_VEGA]',
       ' [C_THETA]', ' [C_RHO]', ' [C_IV]', ' [C_VOLUME]', ' [C_LAST]',
       ' [C_SIZE]', ' [C_BID]', ' [C_ASK]', ' [STRIKE]']]
put_df = df[['[QUOTE_UNIXTIME]', ' [QUOTE_READTIME]', ' [QUOTE_DATE]',
       ' [QUOTE_TIME_HOURS]', ' [UNDERLYING_LAST]', ' [EXPIRE_DATE]',
       ' [EXPIRE_UNIX]', ' [DTE]',' [STRIKE]', ' [P_BID]',
       ' [P_ASK]', ' [P_SIZE]', ' [P_LAST]', ' [P_DELTA]', ' [P_GAMMA]',
       ' [P_VEGA]', ' [P_THETA]', ' [P_RHO]', ' [P_IV]', ' [P_VOLUME]']]

In [2586]:
# Dataframe shape check
print(f"Call Dataframe Shape:{call_df.shape}")
print(f"Put Dataframe Shape:{put_df.shape}")

Call Dataframe Shape:(766, 20)
Put Dataframe Shape:(766, 20)


In [2587]:
# Filter as per quote date
quote_date = '2023-04-20'
call_df = call_df[call_df[" [QUOTE_DATE]"] == quote_date]
put_df = put_df[put_df[" [QUOTE_DATE]"] == quote_date]

In [2588]:
call_df

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],[C_VEGA],[C_THETA],[C_RHO],[C_IV],[C_VOLUME],[C_LAST],[C_SIZE],[C_BID],[C_ASK],[STRIKE]
436,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,1.00000,0.0,0.00000,0.00000,0.00000,,1,3933.50,10 x 10,3919.2,3935.30,200
437,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,1.00000,0.0,0.00000,0.00000,0.00000,,,0.00,10 x 10,3719.3,3735.40,400
438,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,1.00000,0.0,0.00000,0.00000,0.00000,,1,3355.84,10 x 10,3519.4,3535.40,600
439,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,1.00000,0.0,0.00000,0.00000,0.00000,,,0.00,10 x 10,3319.5,3335.50,800
440,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,1.00000,0.0,0.00000,0.00000,0.00000,,175,3146.35,10 x 10,3119.6,3135.60,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,0.00000,0.0,0.00207,-0.02565,0.00020,1.90225,0,0.40,0 x 265,0.0,0.05,6600
762,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,0.00000,0.0,0.00239,-0.02531,0.00024,2.0116,,0.00,0 x 265,0.0,0.05,6800
763,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,0.00042,0.0,0.00227,-0.02499,0.00030,2.11897,,0.00,0 x 265,0.0,0.05,7000
764,1682020800,2023-04-20 16:00,2023-04-20,16,4129.75,2023-04-21,1682107200,1,0.00048,0.0,0.00189,-0.02464,0.00023,2.22138,,0.00,0 x 265,0.0,0.05,7200


In [2514]:
# Dataframe shape check
print(f"Call Dataframe Shape:{call_df.shape}")
print(f"Put Dataframe Shape:{put_df.shape}")

Call Dataframe Shape:(330, 20)
Put Dataframe Shape:(330, 20)


In [2515]:
# Define a function to process and clean the dataframes
def process_dataframe(df, option_type):
    # Drop the columns not required
    if option_type == 'c':
        df = df.drop(columns=[
            '[QUOTE_UNIXTIME]',
            ' [QUOTE_READTIME]',
            ' [QUOTE_DATE]',
            ' [QUOTE_TIME_HOURS]',
            ' [EXPIRE_DATE]',
            ' [EXPIRE_UNIX]',
            ' [C_SIZE]'
        ])
    elif option_type == 'p':
        df = df.drop(columns=[
            '[QUOTE_UNIXTIME]',
            ' [QUOTE_READTIME]',
            ' [QUOTE_DATE]',
            ' [QUOTE_TIME_HOURS]',
            ' [EXPIRE_DATE]',
            ' [EXPIRE_UNIX]',
            ' [P_SIZE]'
        ])
    
    # Rename the columns
    if option_type == 'c':
        df = df.rename(columns={
            ' [UNDERLYING_LAST]': 'Underlying EOD',
            ' [DTE]': 'DTE',
            ' [C_DELTA]': 'Delta',
            ' [C_GAMMA]': 'Gamma',
            ' [C_VEGA]': 'Vega',
            ' [C_THETA]': 'Theta',
            ' [C_RHO]': 'Rho',
            ' [C_IV]': 'IV',
            ' [C_VOLUME]': 'Volm',
            ' [C_LAST]': 'Last',
            ' [C_BID]': 'Bid',
            ' [C_ASK]': 'Ask',
            ' [STRIKE]': 'Strike'
        })
    elif option_type == 'p':
        df = df.rename(columns={
            ' [UNDERLYING_LAST]': 'Underlying EOD',
            ' [DTE]': 'DTE',
            ' [P_DELTA]': 'Delta',
            ' [P_GAMMA]': 'Gamma',
            ' [P_VEGA]': 'Vega',
            ' [P_THETA]': 'Theta',
            ' [P_RHO]': 'Rho',
            ' [P_IV]': 'IV',
            ' [P_VOLUME]': 'Volm',
            ' [P_LAST]': 'Last',
            ' [P_BID]': 'Bid',
            ' [P_ASK]': 'Ask',
            ' [STRIKE]': 'Strike'
        })
    
    # Converting data types to numeric for numeric columns
    columns_to_convert = [
        'Underlying EOD',
        'DTE',
        'Delta',
        'Gamma',
        'Vega',
        'Theta',
        'Rho',
        'IV',
        'Volm',
        'Last',
        'Bid',
        'Ask',
        'Strike'
    ]
    df[columns_to_convert] = df[columns_to_convert].apply(pandas.to_numeric, errors='coerce')
    
    # Adding option_type column
    df['option_type'] = option_type
    
    # Adding ticker column
    def generate_ticker(row):
        if row['option_type'] == 'c':
            return f'SPX 12/01/24 C{row["Strike"]}'
        elif row['option_type'] == 'p':
            return f'SPX 12/01/24 P{row["Strike"]}'
        else:
            return None

    # Adding the 'Ticker' column
    df['Ticker'] = df.apply(generate_ticker, axis=1)
    
    # Adding the Risk-Free Rate column
    df['Risk-Free Rate'] = 0.05
    
    # Remove all rows where the spread is less than 0
    df = df[(df['Ask'] - df['Bid']) >= 0]
    
    # Remove all rows where bid and ask is equal to 0
    df = df[(df['Bid'] != 0) & (df['Ask'] != 0)]
        
    # Calculate the spread
    df["Spread"] = df["Ask"] - df["Bid"]
    
    # Calculate the mid-quote
    df["Mid-Quote"] = (df["Ask"] + df["Bid"]) / 2
    
    # Annualising DTE column
    df['DTE'] = 1/365
    
    return df

In [2516]:
# Process and clean the dataframes
call_df = process_dataframe(call_df, 'c')
put_df = process_dataframe(put_df, 'p')

In [2517]:
# Dataframe shape check
print(f"Call Dataframe Shape:{call_df.shape}")
print(f"Put Dataframe Shape:{put_df.shape}")

Call Dataframe Shape:(237, 18)
Put Dataframe Shape:(174, 18)


In [2518]:
check_df_1 = call_df[call_df['Strike'] > call_df['Underlying EOD']]
check_df_2 = put_df[put_df['Strike'] < put_df['Underlying EOD']]
print(f"OTM Call Dataframe Shape:{check_df_1.shape}")
print(f"OTM Put Dataframe Shape:{check_df_2.shape}")

OTM Call Dataframe Shape:(14, 18)
OTM Put Dataframe Shape:(67, 18)


In [2519]:
# Find the intersection of the "Strike" columns
overlapping_strikes = set(call_df['Strike']).intersection(set(put_df['Strike']))

# Convert the set to a list and sort it
overlapping_strikes = sorted(list(overlapping_strikes))

# Find the range of overlapping observations
if overlapping_strikes:
    min_strike = min(overlapping_strikes)
    max_strike = max(overlapping_strikes)
    print(f"The range of overlapping observations is: {min_strike} to {max_strike}")
else:
    print("There are no overlapping observations in the 'Strike' column.")

# Display the result in a DataFrame format for clarity
overlapping_df = pandas.DataFrame(overlapping_strikes, columns=['Overlapping_Strikes'])
print(overlapping_df)

The range of overlapping observations is: 3750 to 4195
    Overlapping_Strikes
0                  3750
1                  3800
2                  3805
3                  3810
4                  3815
..                  ...
76                 4175
77                 4180
78                 4185
79                 4190
80                 4195

[81 rows x 1 columns]


In [2520]:
# Function to interpolate missing Mid-Quotes
def interpolate_mid_quotes(df, full_strike_range, method='pchip'):
    strikes = df['Strike']
    mid_quotes = df['Mid-Quote']
    
    # Define the interpolator
    if method == 'spline':
        interpolator = UnivariateSpline(strikes, mid_quotes, s=0)
    elif method == 'pchip':
        interpolator = PchipInterpolator(strikes, mid_quotes)
    else:
        raise ValueError("Method should be either 'spline' or 'pchip'")
    
    # Interpolate the Mid-Quotes for the full range of strikes
    interpolated_mid_quotes = interpolator(full_strike_range)
    
    # Create a new DataFrame for the interpolated strikes and Mid-Quotes
    interpolated_df = pandas.DataFrame({
        'Strike': full_strike_range,
        'Interpolated Mid-Quote': interpolated_mid_quotes
    })
    
    return interpolated_df

# Define the full range of overlapping strikes to interpolate over with a partition of 10^-4
full_strike_range = numpy.arange(3500, 4400 + 0.0001, 0.0001)

# Interpolate Mid-Quotes for call options
call_interpolated_df = interpolate_mid_quotes(call_df, full_strike_range, method='spline')

# Interpolate Mid-Quotes for put options
put_interpolated_df = interpolate_mid_quotes(put_df, full_strike_range, method='spline')

In [2521]:
# Merge the dataframes on the 'Strike' column
merged_interpolated_df = pandas.merge(call_interpolated_df, put_interpolated_df, on='Strike', suffixes=('_call', '_put'))

# Calculate the difference between the call and put Mid-quotes
merged_interpolated_df['Difference'] = merged_interpolated_df['Interpolated Mid-Quote_call'] - merged_interpolated_df['Interpolated Mid-Quote_put']

# Find the strike price where the absolute difference is closest to zero
atm_strike = merged_interpolated_df.loc[merged_interpolated_df['Difference'].abs().idxmin(), 'Strike']

print(f"The implied forward ATM strike price is: {atm_strike}")

The implied forward ATM strike price is: 4129.623101273549


In [2522]:
call_df = call_df[call_df['Strike'] > atm_strike]
put_df = put_df[put_df['Strike'] < atm_strike]
call_df = call_df[call_df['Volm'] > 25]
put_df = put_df[put_df['Volm'] > 25]
print(f"OTM Call Dataframe Shape:{check_df_1.shape}")
print(f"OTM Put Dataframe Shape:{check_df_2.shape}")

OTM Call Dataframe Shape:(14, 18)
OTM Put Dataframe Shape:(67, 18)


In [2523]:
merged_df = pandas.concat([call_df, put_df])
merged_df = merged_df.sort_values(by='Strike', ascending=True)
merged_df

,Underlying EOD,DTE,Delta,Gamma,Vega,Theta,Rho,IV,Volm,Last,Bid,Ask,Strike,option_type,Ticker,Risk-Free Rate,Spread,Mid-Quote
583,4129.623101,0.00274,-0.00146,0.00002,0.01785,-0.07462,-0.00057,0.47064,264.0,0.05,0.05,0.10,3750,p,SPX 12/01/24 P3750,0.05,0.05,0.075
593,4129.623101,0.00274,-0.00218,0.00010,0.01935,-0.07522,-0.00002,0.40948,211.0,0.07,0.05,0.10,3800,p,SPX 12/01/24 P3800,0.05,0.05,0.075
595,4129.623101,0.00274,-0.00212,0.00006,0.02039,-0.07475,-0.00039,0.39674,99.0,0.20,0.05,0.10,3810,p,SPX 12/01/24 P3810,0.05,0.05,0.075
598,4129.623101,0.00274,-0.00230,0.00003,0.02061,-0.07505,-0.00077,0.37918,47.0,0.10,0.05,0.10,3825,p,SPX 12/01/24 P3825,0.05,0.05,0.075
600,4129.623101,0.00274,-0.00286,0.00009,0.02151,-0.07520,-0.00037,0.36617,30.0,0.10,0.05,0.10,3835,p,SPX 12/01/24 P3835,0.05,0.05,0.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,4129.623101,0.00274,0.02006,0.00175,0.13757,-0.18493,0.00337,0.09884,2987.0,0.18,0.15,0.25,4175,c,SPX 12/01/24 C4175,0.05,0.10,0.200
669,4129.623101,0.00274,0.01543,0.00138,0.10836,-0.14182,0.00232,0.10221,1096.0,0.10,0.10,0.20,4180,c,SPX 12/01/24 C4180,0.05,0.10,0.150
670,4129.623101,0.00274,0.01068,0.00095,0.07881,-0.09585,0.00151,0.10335,945.0,0.08,0.05,0.15,4185,c,SPX 12/01/24 C4185,0.05,0.10,0.100
671,4129.623101,0.00274,0.00749,0.00071,0.06140,-0.07264,0.00171,0.10561,2067.0,0.07,0.05,0.10,4190,c,SPX 12/01/24 C4190,0.05,0.05,0.075


In [2524]:
import pandas as pd

def filter_unique_mid_quotes(df):
    def filter_monotonic_options(option_df, option_type):
        # Sort by strike price for processing
        option_df = option_df.sort_values(by='Strike', ascending=(option_type == 'p'))
        
        # Initialize the result list
        filtered_rows = []
        seen_mid_quotes = set()
        
        # Iterate through the sorted DataFrame
        for _, row in option_df.iterrows():
            mid_quote = row['Mid-Quote']
            
            # Check if this mid-quote has been seen before
            if mid_quote not in seen_mid_quotes:
                seen_mid_quotes.add(mid_quote)
                filtered_rows.append(row)
        
        # Convert the list of rows back to a DataFrame
        return pd.DataFrame(filtered_rows)
    
    # Split the DataFrame into puts and calls
    puts_df = df[df['option_type'] == 'p']
    calls_df = df[df['option_type'] == 'c']
    
    # Apply the monotonic filtering for puts and calls separately
    filtered_puts = filter_monotonic_options(puts_df, 'p')
    filtered_calls = filter_monotonic_options(calls_df, 'c')
    
    # Combine the filtered puts and calls back together
    filtered_df = pd.concat([filtered_puts, filtered_calls]).sort_index()
    
    return filtered_df.reset_index(drop=True)

# Example usage
# Assuming 'df' is your pandas DataFrame with the specified columns
merged_df = filter_unique_mid_quotes(merged_df)

In [2527]:
call_df = call_df[['Underlying EOD','DTE','Strike','Risk-Free Rate','Mid-Quote','Bid','Ask','Volm', 'option_type']]
put_df = put_df[['Underlying EOD','DTE','Strike','Risk-Free Rate','Mid-Quote','Bid','Ask','Volm', 'option_type']]
merged_df = merged_df[['Underlying EOD','DTE','Strike','Risk-Free Rate','Mid-Quote','Bid','Ask','Volm', 'option_type']]

In [2528]:
merged_df['Quote Date'] = quote_date

In [2529]:
print(f"Merged Dataframe Shape:{merged_df.shape}")

Merged Dataframe Shape:(31, 10)


In [2530]:
merged_df

,Underlying EOD,DTE,Strike,Risk-Free Rate,Mid-Quote,Bid,Ask,Volm,option_type,Quote Date
0,4129.623101,0.00274,3750,0.05,0.075,0.05,0.10,264.0,p,2023-04-20
1,4129.623101,0.00274,3840,0.05,0.100,0.05,0.15,31.0,p,2023-04-20
2,4129.623101,0.00274,3900,0.05,0.125,0.10,0.15,1247.0,p,2023-04-20
3,4129.623101,0.00274,3950,0.05,0.150,0.10,0.20,1039.0,p,2023-04-20
4,4129.623101,0.00274,3970,0.05,0.175,0.15,0.20,230.0,p,2023-04-20
5,4129.623101,0.00274,3995,0.05,0.200,0.15,0.25,186.0,p,2023-04-20
6,4129.623101,0.00274,4020,0.05,0.225,0.20,0.25,134.0,p,2023-04-20
7,4129.623101,0.00274,4045,0.05,0.250,0.20,0.30,161.0,p,2023-04-20
8,4129.623101,0.00274,4075,0.05,0.300,0.25,0.35,900.0,p,2023-04-20
9,4129.623101,0.00274,4085,0.05,0.350,0.30,0.40,1189.0,p,2023-04-20


In [2532]:
# merged_file_path = '/Users/macbook/Downloads/oad_4.xlsx'

# merged_df.to_excel(merged_file_path, index=False)